# Chapter 2: Forward Pricing
This notebook explores the concepts and code implementations related to forward pricing, arbitrage, and the mechanics of different asset classes such as commodities, stocks, bonds, and FX.

# **Table of Contents**  
1. [Forward Pricing Calculator](#forward-pricing-calculator)  
2. [Convenience Yield Imputer(Commodities)](#convenience-yield-imputer)  
3. [Arbitrage Evaluator(Cash & Reverse Cash-and-Carry)](#arbitrage-evaluator)  

# <a id='forward-pricing-calculator'></a>  1. Forward Pricing Calculator
The theoretical **forward price** represents the expected future price of an asset, adjusted for the **cost of carry** (interest, storage, insurance) and **benefits** like dividends or coupon payments. Different asset classes use slightly different formulas.

In [2]:
# Forward Pricing Calculator
def calculate_forward_price(asset_type, spot_price, r, t, **kwargs):
    """
    Parameters:
    - asset_type: "commodity", "stock", "bond", "fx"
    - spot_price: current market price
    - r: risk-free interest rate (annualized)
    - t: time to maturity in years
    - kwargs: asset-specific costs/benefits (e.g., dividends, storage cost, etc.)

    Returns:
    - theoretical forward price
    """
    if asset_type == "commodity":
        storage_cost = kwargs.get("storage_cost",0)
        insurance_cost = kwargs.get("insurance_cost",0)
        return spot_price * (1 + r * t) + (storage_cost + insurance_cost) * t
    
    elif asset_type == "stock":
        dividends = kwargs.get("dividends",0)
        return spot_price * (1 + r * t) - dividends
    
    elif asset_type == "bond":
        coupons = kwargs.get("coupons", 0)
        interest_on_coupons = kwargs.get("interest_on_coupons", 0)
        return spot_price * (1 + r * t) - (coupons + interest_on_coupons)
    
    elif asset_type == "fx":
        foreign_rate = kwargs.get("foreign_rate", 0)
        return spot_price * ((1 + r * t) / (1 + foreign_rate * t))
    
    else:
        raise ValueError("Unsupported asset type")

### **Example Usage**

In [3]:
# Example: Forward price of a stock with dividend of $2
calculate_forward_price("stock", spot_price=100, r=0.05, t=0.5, dividends=2)

100.49999999999999

# <a id='convenience-yield-imputer'></a>  2. Convenience Yield Imputer (Commodities)
The **convenience yield** represents the non-monetary benefit of holding a physical commodity (like availability or liquidity). It is inferred by comparing the actual spot price with the infered one.

In [13]:
# Infers implied convenience yield from observed market forward price
def infer_convenience_yield(market_forward, spot_price, r, t, storage_cost=0, insurance_cost=0):
    """
    Parameters:
    - market_forward: observed market forward price
    - spot_price: current market price
    - r: interest rate
    - t: time to maturity (in years)
    - storage_cost: per unit annual storage cost
    - insurance_cost: per unit annual insurance cost

    Returns:
    - estimated convenience yield
    """
    infered_spot_price = (market_forward - (storage_cost + insurance_cost) * t)/(1 + r * t)
    convenience_yield = spot_price - infered_spot_price
    return round(convenience_yield, 4)

### **Example Usage**

In [14]:
# Example Usage:
infer_convenience_yield(
    market_forward=77.4,
    spot_price=76.25,
    r=0.08,
    t=3/12,
    storage_cost=3,
    insurance_cost=0.6)

1.25

# <a id='arbitrage-evaluator'></a>  3. Arbitrage Evaluator (Cash & Reverse Cash-and-Carry)
This module detects potential arbitrage opportunities.
- Cash-and-Carry involves **buying** the **spot** asset and **selling** a forward contract.

- Reverse Cash-and-Carry involves **shorting** the spot and **buying** a forward.

The key is comparing market forward prices with theoretical values under funding or short borrowing rates.

In [13]:
def arbitrage_evaluator(
    spot_price,
    futures_price,
    interest_rate,
    time,
    expected_dividend=0.0,
    short_rate=None
):
    """
    Evaluate arbitrage using cash-and-carry or reverse cash-and-carry logic.
    Uses short rate in reverse arbitrage if provided.

    Parameters:
    - spot_price: Current spot price of the asset
    - futures_price: Price of the futures contract
    - interest_rate: Annual borrowing rate (decimal)
    - time: Time to expiration (years)
    - expected_dividend: expected dividend till maturity
    - short_rate: Rebate rate for short sellers (decimal, default = None)
    """
    fair_value_cash_carry = spot_price * (1 + interest_rate * time) - expected_dividend
    if short_rate is not None:
        fair_value_reverse = spot_price * (1 + short_rate * time) - expected_dividend
    else:
        fair_value_reverse = fair_value_cash_carry  # fallback

    print(f"Fair Value (Cash-and-Carry): {fair_value_cash_carry:.2f}")
    if short_rate is not None:
        print(f"Fair Value (Reverse Cash-and-Carry using short rate): {fair_value_reverse:.2f}")
    print(f"Actual Futures Price: {futures_price:.2f}\n")

    if futures_price > fair_value_cash_carry:
        print("✅ Cash-and-Carry Arbitrage Opportunity:")
        print("Buy the spot asset and sell the overpriced futures.")
    elif futures_price < fair_value_reverse:
        print("✅ Reverse Cash-and-Carry Arbitrage Opportunity:")
        print("Short the spot asset and buy the underpriced futures.")
    else:
        print("✅ No arbitrage opportunity detected. Markets appear efficient.")

### **Example Usage**

In [14]:
# Example Usage:
arbitrage_evaluator(
    spot_price=67,
    futures_price=69.5,
    interest_rate=0.06,
    short_rate=0.04,
    expected_dividend=0.66,
    time=8/12
)

Fair Value (Cash-and-Carry): 69.02
Fair Value (Reverse Cash-and-Carry using short rate): 68.13
Actual Futures Price: 69.50

✅ Cash-and-Carry Arbitrage Opportunity:
Buy the spot asset and sell the overpriced futures.
